In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import kagglehub

# Download latest version
mpii_human_pose_path = kagglehub.dataset_download('harshpatel66/mpii-human-pose')
print("Path to dataset files:", mpii_human_pose_path)

Path to dataset files: /kaggle/input/mpii-human-pose


In [3]:
! pip install mediapipe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 57.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 13.0 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 18.1.0 which is incompatible.
google-api-core 1.34.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<4.0.0dev,>=3.19.5, but you have protobuf 4.25.5 which is incompatible.
google-cloud-bigtable 2.26.0 requires google-api-core[grpc]<3.0.0dev,>=2.16.0, but you have google-api-core 1.34.1 which is incompatible.
pandas-gbq 0.23.1 requires google-api-core<3.0.0dev,>=2.10.2, but

In [4]:
!pip install pycocotools


In [44]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import json
from scipy.spatial.distance import euclidean

# Initialize MediaPipe Pose
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# For static images (example paths for your setup)
image_dir = '/kaggle/input/mpii-human-pose/mpii_human_pose_v1/images'
json_annotations_path = '/kaggle/input/mpii-human-pose/mpii_annotations.json'
csv_keypoints_path = '/kaggle/input/mpii-human-pose/mpii_human_pose.csv'

# Load CSV and JSON data
ground_truth_df = pd.read_csv(csv_keypoints_path)
with open(json_annotations_path, 'r') as f:
    mpii_annotations = json.load(f)

# Prepare Pose model
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Define the mapping from MediaPipe's 33 keypoints to MPII's 14 keypoints
# Only use the first 14 keypoints from MediaPipe
keypoint_mapping = {
    0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12, 13: 13
    # 33 predicted -> 14 ground truth mapping
}

# Calculate PCK (Percentage of Correct Keypoints)
def calculate_pck(predicted_keypoints, ground_truth_keypoints, image_width, image_height, threshold=0.05):
    threshold_distance = threshold * np.sqrt(image_width**2 + image_height**2)
    
    correct_keypoints = 0
    total_keypoints = 0

    for idx in range(len(predicted_keypoints)):
        pred_x, pred_y = predicted_keypoints[idx]
        if idx >= len(ground_truth_keypoints):
            continue  # Skip if there are more predicted keypoints than ground truth
        
        gt_x, gt_y = ground_truth_keypoints[idx]
        
        if gt_x == -1 or gt_y == -1:  # Ignore missing ground truth points
            continue
        
        total_keypoints += 1
        
        # Calculate Euclidean distance between predicted and ground truth keypoints
        dist = euclidean((pred_x, pred_y), (gt_x, gt_y))
        
        # If distance is less than threshold, consider it correct
        if dist < threshold_distance:
            correct_keypoints += 1
    
    return (correct_keypoints / total_keypoints) * 100 if total_keypoints > 0 else 0

# Calculate Mean Average Precision (mAP)
def calculate_map(predictions, ground_truths, image_width, image_height, thresholds=[ 0.2, 0.3, 0.6]):
    average_precisions = []
    
    for threshold in thresholds:
        pck_values = []
        
        print(f"\nThreshold: {threshold}")
        
        for pred, gt in zip(predictions, ground_truths):
            pred_keypoints = pred['keypoints']
            gt_keypoints = gt['keypoints']
            
            pck = calculate_pck(pred_keypoints, gt_keypoints, image_width, image_height, threshold)
            pck_values.append(pck)
            
            # Print the PCK for this image at the current threshold
        
        # Calculate Average PCK for this threshold
        avg_pck = np.mean(pck_values)
        average_precisions.append(avg_pck)
        
        # Print average PCK at this threshold
        print(f"Average PCK @ {threshold}: {avg_pck:.2f}%")
    
    return np.mean(average_precisions) if len(average_precisions) > 0 else 0

# Process a subset of the dataset (2% of the data)
subset_size = int(len(mpii_annotations) * 0.5)  # 2% of the dataset
annotations_subset = mpii_annotations[:subset_size]

predictions = []
ground_truths = []

# Process images
for idx, annotation in enumerate(annotations_subset):  # Iterate over the 2% subset
    img_path = annotation['img_paths']
    
    
    # Get image dimensions
    image_height, image_width, _ = image.shape
    
    # Process image using MediaPipe Pose model
    results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    
    if not results.pose_landmarks:
        continue
    
    # Prepare predicted keypoints (only use the first 14 keypoints from 33 keypoints)
    predicted_keypoints = [(lm.x * image_width, lm.y * image_height) for i, lm in enumerate(results.pose_landmarks.landmark) if i < 14]  # Only 14 keypoints
    
    # Prepare ground truth keypoints from the CSV
    gt_row = ground_truth_df[ground_truth_df['NAME'] == img_path]
    
    if not gt_row.empty:
        # Extract keypoints from ground truth row
        ground_truth_keypoints = [
            (gt_row[f'r ankle_X'].values[0], gt_row[f'r ankle_Y'].values[0]),
            (gt_row[f'r knee_X'].values[0], gt_row[f'r knee_Y'].values[0]),
            (gt_row[f'r hip_X'].values[0], gt_row[f'r hip_Y'].values[0]),
            (gt_row[f'l hip_X'].values[0], gt_row[f'l hip_Y'].values[0]),
            (gt_row[f'l knee_X'].values[0], gt_row[f'l knee_Y'].values[0]),
            (gt_row[f'l ankle_X'].values[0], gt_row[f'l ankle_Y'].values[0]),
            (gt_row[f'r wrist_X'].values[0], gt_row[f'r wrist_Y'].values[0]),
            (gt_row[f'r elbow_X'].values[0], gt_row[f'r elbow_Y'].values[0]),
            (gt_row[f'r shoulder_X'].values[0], gt_row[f'r shoulder_Y'].values[0]),
            (gt_row[f'l shoulder_X'].values[0], gt_row[f'l shoulder_Y'].values[0]),
            (gt_row[f'l elbow_X'].values[0], gt_row[f'l elbow_Y'].values[0]),
            (gt_row[f'l wrist_X'].values[0], gt_row[f'l wrist_Y'].values[0]),
            (gt_row[f'head top_X'].values[0], gt_row[f'head top_Y'].values[0])  # Use 'head top' keypoints
        ]
    
        # Optionally include nose if necessary
        if 'upper neck_X' in ground_truth_df.columns and 'upper neck_Y' in ground_truth_df.columns:
            ground_truth_keypoints.append((gt_row[f'upper neck_X'].values[0], gt_row[f'upper neck_Y'].values[0]))
        
                
        # Ensure that the predicted and ground truth keypoints are consistent
        if len(predicted_keypoints) == len(ground_truth_keypoints):
            predictions.append({'img_path': img_path, 'keypoints': predicted_keypoints})
            ground_truths.append({'img_path': img_path, 'keypoints': ground_truth_keypoints})
        else:
            print(f"Skipping {img_path} due to mismatch in keypoints count.")
    
# After processing the subset, calculate mAP
print("Calculating Mean Average Precision (mAP)...")
mean_ap = calculate_map(predictions, ground_truths, image_width, image_height)
print(f'Mean Average Precision (mAP): {mean_ap:.4f}')


Calculating Mean Average Precision (mAP)...

Threshold: 0.2
Average PCK @ 0.2: 16.86%

Threshold: 0.3
Average PCK @ 0.3: 29.94%

Threshold: 0.6
Average PCK @ 0.6: 61.36%
Mean Average Precision (mAP): 36.0545


In [29]:
print(ground_truth_df.columns)


Index(['ID', 'NAME', 'r ankle_X', 'r ankle_Y', 'r knee_X', 'r knee_Y',
       'r hip_X', 'r hip_Y', 'l hip_X', 'l hip_Y', 'l knee_X', 'l knee_Y',
       'l ankle_X', 'l ankle_Y', 'pelvis_X', 'pelvis_Y', 'thorax_X',
       'thorax_Y', 'upper neck_X', 'upper neck_Y', 'head top_X', 'head top_Y',
       'r wrist_X', 'r wrist_Y', 'r elbow_X', 'r elbow_Y', 'r shoulder_X',
       'r shoulder_Y', 'l shoulder_X', 'l shoulder_Y', 'l elbow_X',
       'l elbow_Y', 'l wrist_X', 'l wrist_Y', 'Scale', 'Activity', 'Category'],
      dtype='object')


In [45]:
import cv2 as cv
import numpy as np
import pandas as pd
from scipy.spatial.distance import euclidean
import random

# Load CSV and JSON data
csv_keypoints_path = '/kaggle/input/mpii-human-pose/mpii_human_pose.csv'
ground_truth_df = pd.read_csv(csv_keypoints_path)

# Example of body parts we are considering
BODY_PARTS = {
    "Nose": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4,
    "LShoulder": 5, "LElbow": 6, "LWrist": 7, "RHip": 8, "RKnee": 9,
    "RAnkle": 10, "LHip": 11, "LKnee": 12, "LAnkle": 13, "REye": 14,
    "LEye": 15, "REar": 16, "LEar": 17, "Background": 18
}

# Matching the 14 keypoints of interest from the CSV columns:
KEYPOINTS = [
    'r ankle_X', 'r ankle_Y', 'r knee_X', 'r knee_Y', 'r hip_X', 'r hip_Y',
    'l hip_X', 'l hip_Y', 'l knee_X', 'l knee_Y', 'l ankle_X', 'l ankle_Y',
    'r wrist_X', 'r wrist_Y', 'r elbow_X', 'r elbow_Y', 'r shoulder_X', 'r shoulder_Y',
    'l shoulder_X', 'l shoulder_Y', 'l elbow_X', 'l elbow_Y', 'l wrist_X', 'l wrist_Y',
    'head top_X', 'head top_Y', 'upper neck_X', 'upper neck_Y'
]

# Function to calculate PCK
def calculate_pck(predicted_keypoints, ground_truth_keypoints, image_width, image_height, threshold=0.05):
    threshold_distance = threshold * np.sqrt(image_width**2 + image_height**2)
    correct_keypoints = 0
    total_keypoints = 0

    for pred, gt in zip(predicted_keypoints, ground_truth_keypoints):
        if gt is None:  # Skip missing ground truth points
            continue
        total_keypoints += 1
        if pred is not None and euclidean(pred, gt) < threshold_distance:
            correct_keypoints += 1

    return (correct_keypoints / total_keypoints) * 100 if total_keypoints > 0 else 0

# Function to calculate Precision and Recall for each keypoint
def calculate_precision_recall(predicted_keypoints, ground_truth_keypoints, image_width, image_height, threshold=0.2):
    threshold_distance = threshold * np.sqrt(image_width**2 + image_height**2)
    tp, fp, fn = 0, 0, 0
    
    for pred, gt in zip(predicted_keypoints, ground_truth_keypoints):
        if gt is None:  # Skip missing ground truth points
            continue
        
        if pred is not None and euclidean(pred, gt) < threshold_distance:
            tp += 1  # True positive: correctly detected
        elif pred is not None:
            fp += 1  # False positive: detected but incorrect
        else:
            fn += 1  # False negative: not detected

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0

    return precision, recall

# Function to process predictions and ground truths
def process_images_and_calculate_pck(predictions, ground_truths, image_width, image_height, thresholds=[0.2, 0.3, 0.6]):
    pck_values = {threshold: [] for threshold in thresholds}
    precision_values = {threshold: [] for threshold in thresholds}
    recall_values = {threshold: [] for threshold in thresholds}

    for threshold in thresholds:
        for idx, (pred, gt) in enumerate(zip(predictions, ground_truths)):
            if (idx + 1) % 100 == 0:  # Print progress every 10 images
                print(f"Evaluating image {idx + 1} of {len(predictions)}...")

            pred_keypoints = pred['keypoints']
            gt_keypoints = gt['keypoints']

            # Calculate PCK for this threshold
            pck = calculate_pck(pred_keypoints, gt_keypoints, image_width, image_height, threshold)
            pck_values[threshold].append(pck)

            # Calculate Precision and Recall
            precision, recall = calculate_precision_recall(pred_keypoints, gt_keypoints, image_width, image_height, threshold)
            precision_values[threshold].append(precision)
            recall_values[threshold].append(recall)

    # Calculate and print Average Precision (AP) for each threshold
    for threshold in thresholds:
        avg_pck = np.mean(pck_values[threshold]) if pck_values[threshold] else 0
        avg_precision = np.mean(precision_values[threshold]) if precision_values[threshold] else 0
        avg_recall = np.mean(recall_values[threshold]) if recall_values[threshold] else 0

        print(f"PCK at threshold {threshold}: {avg_pck:.2f}")
        print(f"Precision at threshold {threshold}: {avg_precision:.2f}")
        print(f"Recall at threshold {threshold}: {avg_recall:.2f}")

# Calculate mAP
def calculate_map(predictions, ground_truths, thresholds):
    ap_values = []
    
    for threshold in thresholds:
        precision_list = []
        recall_list = []
        
        for pred, gt in zip(predictions, ground_truths):
            pred_keypoints = pred['keypoints']
            gt_keypoints = gt['keypoints']
            
            # Get precision and recall for this keypoint at the threshold
            precision, recall = calculate_precision_recall(pred_keypoints, gt_keypoints, 368, 368, threshold)
            precision_list.append(precision)
            recall_list.append(recall)
        
        # Average precision for the threshold
        ap = np.mean(precision_list) if precision_list else 0
        ap_values.append(ap)

    mAP = np.mean(ap_values)
    print(f"Mean Average Precision (mAP): {mAP:.2f}")
    return mAP

# Path to the TensorFlow model
model_path = '/kaggle/input/graph_opt.pb/tensorflow2/default/1/graph_opt.pb'

# Load the model
net = cv.dnn.readNetFromTensorflow(model_path)

# Image directory and CSV keypoints path
image_dir = '/kaggle/input/mpii-human-pose/mpii_human_pose_v1/images'
csv_keypoints_path = '/kaggle/input/mpii-human-pose/mpii_human_pose.csv'

# Reading the images and matching ground truth
predictions = []
ground_truths = []

# Get 2% of the dataset
sample_size = int(len(ground_truth_df) * 0.5)
sampled_indices = random.sample(range(len(ground_truth_df)), sample_size)

# Iterate through the subset of 2% images
for idx, row in ground_truth_df.iloc[sampled_indices].iterrows():
    img_path = row['NAME']
    image = cv.imread(f"{image_dir}/{img_path}")
    
    if image is None:
        print(f"Image {img_path} could not be loaded, skipping...")
        continue

    # Get image dimensions
    image_height, image_width, _ = image.shape

    # Process the image with the model
    net.setInput(cv.dnn.blobFromImage(image, 1.0, (368, 368), (127.5, 127.5, 127.5), swapRB=True, crop=False))
    out = net.forward()
    out = out[:, :19, :, :]  # We need the first 19 elements for pose estimation

    points = []
    for i in range(14):  # Extract first 14 keypoints
        heat_map = out[0, i, :, :]
        _, conf, _, point = cv.minMaxLoc(heat_map)
        x = (image_width * point[0]) / out.shape[3]
        y = (image_height * point[1]) / out.shape[2]
        points.append((int(x), int(y)) if conf > 0.2 else None)

    # Prepare predicted keypoints
    predictions.append({'img_path': img_path, 'keypoints': points})

    # Prepare ground truth keypoints (from CSV)
    ground_truth_keypoints = []
    for keypoint in KEYPOINTS:
        if pd.isna(row[keypoint]):
            ground_truth_keypoints.append(None)
        else:
            ground_truth_keypoints.append((row[keypoint], row[keypoint.replace('X', 'Y')]))

    ground_truths.append({'img_path': img_path, 'keypoints': ground_truth_keypoints})

# Now calculate the PCK and mAP for different thresholds
print("Starting evaluation...")
process_images_and_calculate_pck(predictions, ground_truths, image_width, image_height, thresholds=[0.2, 0.3, 0.6])

# Calculate mAP
calculate_map(predictions, ground_truths, thresholds=[ 0.2, 0.3, 0.6])


Starting evaluation...
Evaluating image 100 of 8686...
Evaluating image 200 of 8686...
Evaluating image 300 of 8686...
Evaluating image 400 of 8686...
Evaluating image 500 of 8686...
Evaluating image 600 of 8686...
Evaluating image 700 of 8686...
Evaluating image 800 of 8686...
Evaluating image 900 of 8686...
Evaluating image 1000 of 8686...
Evaluating image 1100 of 8686...
Evaluating image 1200 of 8686...
Evaluating image 1300 of 8686...
Evaluating image 1400 of 8686...
Evaluating image 1500 of 8686...
Evaluating image 1600 of 8686...
Evaluating image 1700 of 8686...
Evaluating image 1800 of 8686...
Evaluating image 1900 of 8686...
Evaluating image 2000 of 8686...
Evaluating image 2100 of 8686...
Evaluating image 2200 of 8686...
Evaluating image 2300 of 8686...
Evaluating image 2400 of 8686...
Evaluating image 2500 of 8686...
Evaluating image 2600 of 8686...
Evaluating image 2700 of 8686...
Evaluating image 2800 of 8686...
Evaluating image 2900 of 8686...
Evaluating image 3000 of 8686

0.24515486698597452